In [2]:
# 多输入通道和多输出通道

# 对于真实数据的维度经常更高，例如彩色图像在高和宽两个维度外还有RGB（红绿蓝）3个通道
# 包含多个输入输出通道的卷积核


In [3]:
from imp import reload
import torch
from torch import nn
import sys

sys.path.append("..")
import d2l_pytorch.d2l as d2l
reload(d2l)




D:\Temp\ipykernel_20924\4074691625.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


<module 'd2l_pytorch.d2l' from '..\\d2l_pytorch\\d2l.py'>

In [4]:
def corr2d_multi_in(X, K):
  # 沿着X和K的第0维（通道维度）分别计算再想加
  res = d2l.corr2d(X[0, :, :], K[0, :, :])
  for i in range(1, X.shape[0]):
    res += d2l.corr2d(X[i, :, :], K[i, :, :])

  return res


In [5]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]], [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]],[[1, 2], [3, 4]]])

corr2d_multi_in(X,K)


tensor([[ 56.,  72.],
        [104., 120.]])

In [6]:
# 多输出通道不论输出通道是多少，输出通道都为1


In [7]:
def corr2d_multi_in_out(X, K):
  return torch.stack([corr2d_multi_in(X, k) for k in K])

In [8]:
K = torch.stack([K, K+1, K+2])
K.shape

torch.Size([3, 2, 2, 2])

In [9]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [10]:
def corr2d_multi_in_out_1x1(X, K):
  c_i, h, w = X.shape
  c_o = K.shape[0]
  X = X.view(c_i, h * w)
  K = K.view(c_o, c_i)
  Y = torch.mm(K, X)
  return Y.view(c_o, h, w)

In [11]:
X = torch.rand(3,3,3)
K = torch.rand(2,3,1,1)
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
(Y1 - Y2).norm().item() < 1e-6

True

In [13]:
X.shape
K.shape[0]

2

In [14]:
# 池化层 
def pool2d(X, pool_size, mode = "max"):
  X = X.float()
  p_h, p_w = pool_size
  Y = torch.zeros(X.shape[0]-p_h + 1, X.shape[1] - p_w + 1)

  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      if mode == "max":
        Y[i, j] = X[i: i+ p_h, j: j+p_w].max()
      elif mode == "avg":
        Y[i, j] = X[i: i+ p_h, j:j+p_w].mean()
  
  return Y

In [15]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2,2))

tensor([[4., 5.],
        [7., 8.]])

In [16]:
pool2d(X, (2,2), "avg")

tensor([[2., 3.],
        [5., 6.]])

In [19]:
X = torch.arange(16, dtype= torch.float).view((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [20]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [21]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [22]:
pool2d = nn.MaxPool2d((2,4), padding=(1, 2), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [23]:
# 多通道池化
X = torch.cat((X, X+1), dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [24]:
pool2d = nn.MaxPool2d(3, padding = 1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

# 小结

* 最大池化和平均池化分别取池化窗口中输入元素的最大值和平均值作为输出
* 池化层的一个主要作用是缓解卷积层对位置的过度敏感性
* 可以指定池化层的填充和步幅
* 池化层的输出通道数跟输入通道数相同